# 10 Minutes to cuDF and CuPy

This notebook provides introductory examples of how you can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations).

In [1]:
import timeit

import cupy as cp
import cudf

### Converting a cuDF DataFrame to a CuPy Array

If we want to convert a cuDF DataFrame to a CuPy ndarray, There are multiple ways to do it:

1. We can use the [dlpack](https://github.com/dmlc/dlpack) interface.

2. We can also use `DataFrame.values`.

3. We can also convert via the [CUDA array interface](https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html) by using cuDF's `as_gpu_matrix` and CuPy's `asarray` functionality.

In [2]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%timeit arr_cupy = cp.fromDlpack(df.to_dlpack())
%timeit arr_cupy = df.values
%timeit arr_cupy = cp.asarray(df.as_gpu_matrix())

48.6 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
222 µs ± 800 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
221 µs ± 590 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
arr_cupy = cp.fromDlpack(df.to_dlpack())
arr_cupy

array([[    0,   500,  1000],
       [    1,   501,  1001],
       [    2,   502,  1002],
       ...,
       [ 9997, 10497, 10997],
       [ 9998, 10498, 10998],
       [ 9999, 10499, 10999]])

### Converting a cuDF Series to a CuPy Array

There are also multiple ways to convert a cuDF Series to a CuPy array:

1. We can pass the Series to `cupy.asarray` as cuDF Series exposes [`__cuda_array_interface__`](https://docs-cupy.chainer.org/en/stable/reference/interoperability.html).
2. We can leverage the dlpack interface `to_dlpack()`. 
3. We can also use `Series.values` 


In [4]:
col = 'a'

%timeit cola_cupy = cp.asarray(df[col])
%timeit cola_cupy = cp.fromDlpack(df[col].to_dlpack())
%timeit cola_cupy = df[col].values

20.8 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
55.2 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
76.8 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
cola_cupy = cp.asarray(df[col])
cola_cupy

array([   0,    1,    2, ..., 9997, 9998, 9999])

From here, we can proceed with normal CuPy workflows, such as reshaping the array, getting the diagonal, or calculating the norm.

In [6]:
reshaped_arr = cola_cupy.reshape(50, 200)
reshaped_arr

array([[   0,    1,    2, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  397,  398,  399],
       [ 400,  401,  402, ...,  597,  598,  599],
       ...,
       [9400, 9401, 9402, ..., 9597, 9598, 9599],
       [9600, 9601, 9602, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9997, 9998, 9999]])

In [7]:
reshaped_arr.diagonal()

array([   0,  201,  402,  603,  804, 1005, 1206, 1407, 1608, 1809, 2010,
       2211, 2412, 2613, 2814, 3015, 3216, 3417, 3618, 3819, 4020, 4221,
       4422, 4623, 4824, 5025, 5226, 5427, 5628, 5829, 6030, 6231, 6432,
       6633, 6834, 7035, 7236, 7437, 7638, 7839, 8040, 8241, 8442, 8643,
       8844, 9045, 9246, 9447, 9648, 9849])

In [8]:
cp.linalg.norm(reshaped_arr)

array(577306.967739)

### Converting a CuPy Array to a cuDF DataFrame

We can also convert a CuPy ndarray to a cuDF DataFrame. Like before, there are multiple ways to do it:

1. We can use the [dlpack](https://github.com/dmlc/dlpack) interface.

2. We can also directly use the `DataFrame` constructor

In [9]:
%timeit reshaped_df = cudf.from_dlpack(reshaped_arr.toDlpack())
%timeit reshaped_df = cudf.DataFrame(reshaped_arr)

5.02 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
reshaped_df = cudf.from_dlpack(reshaped_arr.toDlpack())
reshaped_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
2,2,3,4,5,6,7,8,9,10,11,...,192,193,194,195,196,197,198,199,200,201
3,3,4,5,6,7,8,9,10,11,12,...,193,194,195,196,197,198,199,200,201,202
4,4,5,6,7,8,9,10,11,12,13,...,194,195,196,197,198,199,200,201,202,203


### Converting a CuPy Array to a cuDF Series

To convert an array to a Series, we can directly pass the array to the `Series` constructor.

In [11]:
cudf.Series(reshaped_arr.diagonal().toDlpack()).head()

0      0
1    201
2    402
3    603
4    804
dtype: int64

### Interweaving CuDF and CuPy for Smooth PyData Workflows

RAPIDS libraries and the entire GPU PyData ecosystem are developing quickly, but sometimes a one library may not have the functionality you need. One example of this might be taking the row-wise sum (or mean) of a Pandas DataFrame. cuDF's support for row-wise operations isn't mature, so you'd need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns (which cuDF wouldn't perform well with) depending on your data's shape, and writing a UDF can be time intensive.

By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. Let's take the row-wise sum of our previously reshaped cuDF DataFrame.

In [12]:
reshaped_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
2,2,3,4,5,6,7,8,9,10,11,...,192,193,194,195,196,197,198,199,200,201
3,3,4,5,6,7,8,9,10,11,12,...,193,194,195,196,197,198,199,200,201,202
4,4,5,6,7,8,9,10,11,12,13,...,194,195,196,197,198,199,200,201,202,203


We can just transform it into a CuPy array and use the `axis` argument of `sum`.

In [13]:
new_arr = cp.fromDlpack(reshaped_df.to_dlpack())
new_arr.sum(axis=1)

array([19900, 20100, 20300, 20500, 20700, 20900, 21100, 21300, 21500,
       21700, 21900, 22100, 22300, 22500, 22700, 22900, 23100, 23300,
       23500, 23700, 23900, 24100, 24300, 24500, 24700, 24900, 25100,
       25300, 25500, 25700, 25900, 26100, 26300, 26500, 26700, 26900,
       27100, 27300, 27500, 27700, 27900, 28100, 28300, 28500, 28700,
       28900, 29100, 29300, 29500, 29700])

With just that single line, we're able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed.

### Converting a cuDF DataFrame to a CuPy Sparse Matrix

We can also convert a DataFrame or Series to a CuPy sparse matrix. We might want to do this if downstream processes expect CuPy sparse matrices as an input.

The sparse matrix data structure is defined by three dense arrays. We'll define a small helper function for cleanliness.

In [14]:
def cudf_to_cupy_sparse_matrix(data, sparseformat='column'):
    """Converts a cuDF object to a CuPy Sparse Column matrix.
    """
    if sparseformat not in ('row', 'column',):
        raise ValueError("Let's focus on column and row formats for now.")
    
    _sparse_constructor = cp.sparse.csc_matrix
    if sparseformat == 'row':
        _sparse_constructor = cp.sparse.csr_matrix

    return _sparse_constructor(cp.fromDlpack(data.to_dlpack()))

We can define a sparsely populated DataFrame to illustrate this conversion to either sparse matrix format.

In [15]:
df = cudf.DataFrame()
nelem = 10000
nonzero = 1000
for i in range(20):
    arr = cp.random.normal(5, 5, nelem)
    arr[cp.random.choice(arr.shape[0], nelem-nonzero, replace=False)] = 0
    df['a' + str(i)] = arr

In [16]:
df.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,-4.575138,0.0,0.000000,0.0,11.756025,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.000000,0.0,10.082001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.608089,0.0,0.0,0.0,0.0
3,0.000000,0.0,1.225759,0.0,0.000000,0.0,13.898460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [17]:
sparse_data = cudf_to_cupy_sparse_matrix(df)
sparse_data

From here, we could continue our workflow with a CuPy sparse matrix.

For a full list of the functionality built into these libraries, we encourage you to check out the API docs for [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [CuPy](https://docs-cupy.chainer.org/en/stable/index.html).